In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import scipy.optimize as optimize

## Toy example for exponential

In [ ]:
def NLL_from_data(lambda_,data):
    log_pdf = np.log(lambda_) - lambda_*data
    log_likelihood = np.sum(log_pdf)
    return -log_likelihood


def NLL_from_pdf(pdf):
    log_pdf = np.log(pdf)
    log_likelihood = np.sum(log_pdf)
    return -log_likelihood

In [ ]:
np.random.seed(42)

lambda_ = 3.0
size=100
data = stats.expon.rvs(scale=1/lambda_,size=size) #draw 'size' datapoints from expon dist with given parameter
pdf = stats.expon.pdf(data,scale=1/lambda_) #calculate probs of the datapoints

print(NLL_from_data(lambda_=lambda_,data=data),NLL_from_pdf(pdf))

In [ ]:

#data gives the arguments to be fixed
#minimise call expects that the first argument is one to be optimised!
p0=[1.0]
bounds = [(1e-6,None)]
options = {'maxiter':1000}
est_1 = (optimize.minimize(NLL_from_data,x0=p0,args=(data,),method='L-BFGS-B',bounds=bounds,options=options)).x[0]

## Truncated exponential fit

In [ ]:

#taken from 'normed fit data' in other file
DATA = [np.array([8.94813851e-01, 1.05365097e-01, 0, 0,
        0, 0]),
 np.array([7.14377891e-01, 2.14385034e-01, 7.15299320e-02, 0,
        0, 0]),
 np.array([6.55267360e-01, 2.06996136e-01, 1.38031332e-01, 0,
        0, 0])]

years = ['23-24', '24-25', '25-26', '26-27', '27-28', '28-29']
mapped_years = np.arange(1,len(years)+1) #this x range is quite arbitrary
assert len(years)==len(mapped_years)

In [ ]:
def discret_exp_dist(lambda_,start=0,end=1,bin_width=0.1):
    n_bins = int(((end-start)/bin_width)) #number of values in pmf
    res=n_bins*100 #100 pdf values for each bin
    dx = (end-start)/res
    data = np.linspace(start,end,res)
    pdf = stats.expon.pdf(data,scale=1/lambda_)
    pmf = np.zeros(n_bins)

    for i in range(n_bins):
        stt_idx = int((i*res)/n_bins)
        end_idx = int((((i+1)/n_bins)*res))
        prob_mass = np.sum(pdf[stt_idx:end_idx]*dx) #integrate
        pmf[i]=prob_mass

    return pmf


def truncated_exp_dist_pdf(lambda_,t,start=0,stop=1,size=100):
    '''
    Generate a truncated exponential distribution (continous)
    '''
    data = np.linspace(start=start,stop=stop,num=size)
    threshold_idx = int((t/(stop-start))*size)
    data_t = data[:threshold_idx]

    expon_pdf_t = stats.expon.pdf(x=data_t,scale=1/lambda_) #pdf from start -> threshold
    norm_factor = stats.expon.cdf(t,scale=1/lambda_) - stats.expon.cdf(start,scale=1/lambda_)
    truncated_pdf = expon_pdf_t/norm_factor
    
    rem_pdf = np.zeros(len(data)-len(data_t))

    full_pdf = np.concatenate([truncated_pdf,rem_pdf])

    return data,full_pdf

def truncated_exp_dist_pmf(pdf,data,t,bin_width=0.1):

    first_zero_idx = np.where(pdf==0)[0][0]
    start,stop=data[0],data[-1]

    #ensure that bins do not straddle non-zero part of pmf and zero part
    div = (t-start)/bin_width
    assert div%1==0 

    bin_lbs = np.arange(start,stop,bin_width)
    bin_ubs = bin_lbs+bin_width
    bin_mps = (bin_lbs+bin_ubs)/2

    n_bins = int((stop-start)/bin_width)
    res = len(pdf) #number of pdf values
    dx = (stop-start)/res
    pmf = np.zeros(n_bins)

    for i in range(n_bins):
        stt_idx = int((i*res)/n_bins)
        end_idx = int((((i+1)/n_bins)*res))
        prob_mass = np.sum(pdf[stt_idx:end_idx]*dx)
        pmf[i]=prob_mass
    
    return bin_mps,pmf

def truncated_exp_dist_pmf_2(pdf,data,t,bin_centers,bin_width):
    bin_lbs = bin_centers-(bin_width/2)
    bin_ubs = bin_centers+(bin_width/2)

    dx=np.round(np.mean(np.diff(data)),np.log10(len(data)));print(dx)


    pmf = np.zeros(len(bin_centers))



start=0
stop=1
lambda_=1.0
t=0.5
bin_width=0.1
bin_centers = np.linspace(0.1,1.1,len(years)) #mapped years

data,pdf = truncated_exp_dist_pdf(lambda_=lambda_,t=t,start=start,stop=stop)
bin_mps,pmf = truncated_exp_dist_pmf(pdf,data,t,bin_width=bin_width)

#plot
plt.bar(bin_mps,pmf,width=bin_width,edgecolor='black')
plt.xticks(bin_mps)
plt.xt

In [ ]:

bin_width=0.2
bin_centers = np.linspace(0.1,1.1,len(years)) #mapped years
bin_lbs = bin_centers-(bin_width/2)
bin_ubs = bin_centers+(bin_width/2)
bin_bounds = np.array(list(zip(bin_lbs,bin_ubs)))

true_pmf = DATA[2]
threshold_bin = bin_centers[(np.where(true_pmf==0)[0][0])]
threshold = threshold_bin-(bin_width/2) #states where pdf should go to 0


start=0
stop=2
lambda_=4.5
t=threshold
size=1000
data,pdf = truncated_exp_dist_pdf(lambda_=lambda_,t=t,start=start,stop=stop,size=size)
dx=np.round(np.mean(np.diff(data)),int(np.log10(len(data)))) #assuming data is of for 10^n

assert(np.round(sum(pdf*dx),1)==1) #check that prob mass sums to 1


pred_pmf = np.zeros(len(bin_centers))

for idx,bin_bound in enumerate(bin_bounds):
    lb,ub=bin_bound
    idx_a=np.argmin(np.abs(data-lb))
    idx_b=np.argmin(np.abs(data-ub)); print(idx_a,idx_b)
    sliced_pdf = pdf[idx_a:idx_b]
    prob_mass = np.sum(pdf[idx_a:idx_b]*dx)
    pred_pmf[idx]=prob_mass

fig,ax=plt.subplots()
ax.bar(bin_centers,true_pmf,width=bin_width,edgecolor='black',label='true pmf',alpha=0.6)
ax.bar(bin_centers,pred_pmf,width=bin_width,edgecolor='black',label='pred pmf',alpha=0.6)
ax.set_xticks(bin_centers)
ax.legend()


In [ ]:
from scipy.optimize import minimize


start=0.1 #start at 0.1 just to see pmf on a bar chart easier
stop=1.1
mapped_years = np.linspace(start,stop,len(years))

def MSE_fit(lambda_,t,probs):
    '''
    We pass (t,probs) and search over lambda_
    '''

def NLL(lambda_,t,data):
    '''
    We are passing (t,data) and need to search over lambda
    '''
    return None

def compute_predicted_pmf(lambda_,t):
    predicted_pdf = truncated_exp_dist_pdf(lambda_=lambda_,
                                t=t,start=0,stop=2,size=1000) #generate truncated pdf
    
    predicted_pmf = None

def MSE_loss(true_pmf):
    predicted_pmf = None
    return np.mean((predicted_pmf-true_pmf)**2)
    
#for 2022 data
true_pmf = DATA[0]
year_threshold_idx = np.where(true_pmf==0)[0][0]
year_t=years[year_threshold_idx]
mapped_year_t = mapped_years[year_threshold_idx]


initial_params = [1.0]
result = minimize()
